In [1]:
!pip install fasthugs

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.4 requires wrapt~=1.12.1, but you have wrapt 1.14.1 which is incompatible.
tensorflow-transform 1.8.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
te

In [2]:
from transformers import AutoModelForSequenceClassification
from fastai.text.all import *
from fastai.callback.wandb import *

from fasthugs.learner import TransLearner
from fasthugs.data import TransformersTextBlock, TextGetter, get_splits, PreprocCategoryBlock

from datasets import load_dataset, concatenate_datasets

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(321)

In [3]:
ds_name = 'glue'
model_name = "AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1"

max_len = 512
bs = 32
val_bs = bs*2

lr = 3e-5

In [4]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
def validate_task():
    assert task in GLUE_TASKS

In [5]:
from fastai.metrics import MatthewsCorrCoef, F1Score, PearsonCorrCoef, SpearmanCorrCoef

In [6]:
glue_metrics = {
    'cola':[MatthewsCorrCoef()],
    'sst2':[accuracy],
    'mrpc':[F1Score(), accuracy],
    'stsb':[PearsonCorrCoef(), SpearmanCorrCoef()],
    'qqp' :[F1Score(), accuracy],
    'mnli':[accuracy],
    'qnli':[accuracy],
    'rte' :[accuracy],
    'wnli':[accuracy],
}

In [7]:
task = 'wnli'
validate_task()

In [8]:
ds = load_dataset(ds_name, task)

Generating train split:   0%|          | 0/635 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/wnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
ds.keys()

dict_keys(['train', 'validation', 'test'])

In [10]:
len(ds['train']), len(ds['validation'])

(635, 71)

In [11]:
train_idx, valid_idx = get_splits(ds)
valid_idx

(#71) [635,636,637,638,639,640,641,642,643,644...]

In [12]:
train_ds = concatenate_datasets([ds['train'], ds['validation']])

In [13]:
train_ds[0]

{'sentence1': 'I stuck a pin through a carrot. When I pulled the pin out, it had a hole.',
 'sentence2': 'The carrot had a hole.',
 'label': 1,
 'idx': 0}

In [14]:
vocab = train_ds.features['label'].names
dblock = DataBlock(blocks = [TransformersTextBlock(pretrained_model_name=model_name), PreprocCategoryBlock(vocab)],
                   get_x=TextGetter('sentence1', 'sentence2'),
                   get_y=ItemGetter('label'),
                   splitter=IndexSplitter(valid_idx))

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [15]:
%%time
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=val_bs)

CPU times: user 3.56 s, sys: 1.79 s, total: 5.35 s
Wall time: 9.84 s


In [16]:
dls.show_batch(max_n=5)

,text,text_,category
0,"I stuck a pin through a carrot. When I pulled the pin out, it had a hole.",The carrot had a hole.,entailment
1,The police arrested all of the gang members. They were trying to run the drug trade in the neighborhood.,The gang members were trying to run the drug trade in the neighborhood.,entailment
2,Susan knows all about Ann's personal problems because she is indiscreet.,Ann is indiscreet.,entailment
3,"My meeting started at 4:00 and I needed to catch the train at 4:30, so there wasn't much time. Luckily, it was short, so it worked out.","The meeting was short, so it worked out.",entailment
4,"This morning, Joey built a sand castle on the beach, and put a toy flag in the highest tower, but this afternoon the wind knocked it down.",This afternoon the wind knocked The flag down.,entailment


In [17]:
import wandb

WANDB_NAME = f'{ds_name}-{task}-{model_name}'
GROUP = f'{ds_name}-{task}-{model_name}-{lr:.0e}'
NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'radam']

In [18]:
#wandb.init(reinit=True, project="fasthugs", entity="fastai_community",
#           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
metrics = glue_metrics[task]
learn = TransLearner(dls, model, metrics=metrics).to_fp16()

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_p

In [20]:
cbs = []
learn.fit_one_cycle(10, lr, cbs=cbs)

epoch,train_loss,valid_loss,accuracy,time
0,0.696667,0.703146,0.436620,00:07
1,0.697119,0.689562,0.563380,00:06
2,0.698606,0.689871,0.563380,00:06
3,0.696312,0.687837,0.563380,00:06
4,0.696849,0.688602,0.563380,00:06
5,0.696669,0.692276,0.563380,00:06
6,0.695071,0.692225,0.563380,00:06
7,0.693708,0.690228,0.563380,00:06
8,0.693920,0.691264,0.563380,00:06
9,0.693152,0.691177,0.563380,00:06


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [21]:
learn.show_results()

,text,text_,category,category_
0,The drain is clogged with hair. It has to be cleaned.,The hair has to be cleaned.,not_entailment,not_entailment
1,I'm sure that my map will show this building; it is very famous.,The building is very famous.,entailment,not_entailment
2,"Beth didn't get angry with Sally, who had cut her off, because she stopped and apologized.",Sally stopped and apologized.,entailment,not_entailment
3,"When Tommy dropped his ice cream, Timmy giggled, so father gave him a stern look.",Father gave Timmy a stern look.,entailment,not_entailment
4,"Bernard, who had not told the government official that he was less than 21 when he filed for a homestead claim, did not consider that he had done anything dishonest. Still, anyone who knew that he was 19 years old could take his claim away from him.",Anyone who knew that he was 19 years old could take his claim away from anyone.,not_entailment,not_entailment
5,Bob collapsed on the sidewalk. Soon he saw Carl coming to help. He was very ill.,Carl was very ill.,not_entailment,not_entailment
6,The drain is clogged with hair. It has to be removed.,The drain has to be removed.,not_entailment,not_entailment
7,Mr. Taylor was a man of uncertain temper and his general tendency was to think that David was a poor chump and that whatever step he took in any direction on his own account was just another proof of his innate idiocy.,Any direction on his own account was just another proof of David's innate idiocy.,entailment,not_entailment
8,"Everyone really loved the oatmeal cookies; only a few people liked the chocolate chip cookies. Next time, we should make fewer of them.",We should make fewer of the oatmeal cookies.,not_entailment,not_entailment
